Notebook that trains an LSTM to generate tweets based on a given emojis

First we load the data

In [436]:
import numpy as np
import pandas as pd
import data_load_utils as util

tweets = util.filter_tweets_min_count(
    util.read_tweet_data('data/emojis_homemade.csv'),
    min_count=1000)

tweets['text'] = util.filter_text_for_handles(tweets['text'])


/home/nickdbn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


AttributeError: module 'pandas' has no attribute 'loc'

In [437]:
tweets

       Unnamed: 0                                               text  emoji
0               0  RT [VID] 181023 - Foi adicionada a letra “D” n...      ©
1               1  RT 181023 Kris Wu Studio update (3/3)\n\nLegen...      💫
2               2  RT Now you are watching Indian SuperStar with ...      😎
3               3                                    dats for keeps       💛
6               6                             Holy shit no I think.       😩
7               7                  RT army, follow who retweet this       👑
8               8  RT Simply K-Pop harddrive dump\n\n#베리굿 #BerryG...      🤩
9               9  Happy birthday nellie hope you have a fantasti...      ❤
10             10  RT "I have passed through fire"\n\nThank you f...      💕
12             12  RT Incredible to be involved in the making of ...      😱
13             13  RT Too much feels \n\n#BTS #MPN #BTSARMY https...      😢
14             14  RT #PL TOP SCORERS \n\n7 - \n6 - the goods  ht...      🔥
15          

In [438]:
tweets.iloc[0,:]

Unnamed: 0                                                    0
text          RT [VID] 181023 - Foi adicionada a letra “D” n...
emoji                                                         ©
Name: 0, dtype: object

In [439]:
tweets.loc[:,'text', 'emoji']

IndexingError: Too many indexers

In [440]:
tweets.loc[:,['text', 'emoji']]

                                                     text  emoji
0       RT [VID] 181023 - Foi adicionada a letra “D” n...      ©
1       RT 181023 Kris Wu Studio update (3/3)\n\nLegen...      💫
2       RT Now you are watching Indian SuperStar with ...      😎
3                                         dats for keeps       💛
6                                  Holy shit no I think.       😩
7                       RT army, follow who retweet this       👑
8       RT Simply K-Pop harddrive dump\n\n#베리굿 #BerryG...      🤩
9       Happy birthday nellie hope you have a fantasti...      ❤
10      RT "I have passed through fire"\n\nThank you f...      💕
12      RT Incredible to be involved in the making of ...      😱
13      RT Too much feels \n\n#BTS #MPN #BTSARMY https...      😢
14      RT #PL TOP SCORERS \n\n7 - \n6 - the goods  ht...      🔥
15      [DEPASCAL] RUSSEL HOODIE BLUE\n https://t.co/C...      🔗
16      The joys of having a make up artist as your be...      😂
18                      R

In [441]:
my_dict = {'text': "red and yellow and pink and green, orange and purple and blue, I can sing a rainbow, sing a rainbow, sing a rainbow too",
               'emoji': ":rainbow:"}
my_series = pd.Series(my_dict)

In [442]:
my_series

text     red and yellow and pink and green, orange and ...
emoji                                            :rainbow:
dtype: object

In [490]:
from importlib import reload
util = reload (util)

In [491]:
# Get the list of strings (x) and next characters (y) from a single tweet
series, next_chars = util.get_series_data_from_tweet (my_series)





# in the full code, about here we'd concatenate all the series from ALL tweets


In [492]:
my_dict = {'text':
           ["red and yellow and pink and green, orange and purple and blue, I can sing a rainbow, sing a rainbow, sing a rainbow too",
           "sweet dreams are made of this, who am I to disagree, travel the world and the even seas, every body's looking for someone"],
           'emoji':
           [":rainbow:",
           ":gay_pride_flag:"]}
my_data = pd.DataFrame(my_dict)

In [493]:
my_data['emoji'][1]

':gay_pride_flag:'

In [494]:
my_data

                                                text             emoji
0  red and yellow and pink and green, orange and ...         :rainbow:
1  sweet dreams are made of this, who am I to dis...  :gay_pride_flag:

In [495]:
# apply the function to split each tweet into multiple windows of 40 chars and
# a corresponding n_char
# spits out a list of (x, y) tuples which is a real headache but we can fix it

big_mess = my_data.apply(
    lambda x: util.get_series_data_from_tweet(x, window_size=40, step=3), axis=1)

In [496]:
(x_tuple, y_tuple) = zip(*big_mess) # unzips the tuples into separate tuples of x, y

In [497]:
# get the universal character set and the corresponding index
chars_univ, char_idx_univ = util.get_universal_chars_list()

Unique chars:  78


In [498]:
np.asarray(x_tuple)
np.asarray(y_tuple)

array([[' ', 'd', 'n', 'y', 'l', ' ', 'd', 'i', ' ', 'd', 'r', 'n', 'o',
        'n', ' ', 'd', 'u', 'l', 'a', ' ', 'u', ' ', 'c', ' ', 'n', 'a',
        'a', 'b', ',', 'i', ' ', 'r', 'n', 'w', 's', 'g', ' ', 'i', 'o',
        't'],
       ['w', 't', 'r', 'm', 'a', ' ', 'd', 'o', 't', 's', 'w', ' ', ' ',
        't', 'd', 'a', 'e', ' ', 'a', 'l', 'h', 'w', 'l', 'a', ' ', 'e',
        'v', ' ', 'a', ' ', 'e', ' ', 'd', 's', 'o', 'i', ' ', 'r', 'o',
        'o']], dtype='<U1')

In [499]:
x_bool = pd.Series(x_tuple).apply(util.get_x_bool_array,
                                  args=(chars_univ, char_idx_univ))

y_bool = pd.Series(y_tuple).apply(util.get_y_bool_array,
                                  args=(chars_univ, char_idx_univ))

In [502]:

x_dims = (len(x_bool),           # indexes over tweets
          x_bool[0].shape[0],    # indexes over different sentence windows ((160 - window_size) / step = 40)
          x_bool[0].shape[1],    # indexes over characters in the window (window_size = 40)
          x_bool[0].shape[2])    #x_bool[0].shape[2]))    # one-hot encoding for each character (78)

y_dims = (len(y_bool),           # indexes over tweets
          y_bool[0].shape[0],    # indexes over different sentence windows (40)
          y_bool[0].shape[1])    # one-hot encoding for each character (78)

In [503]:
# allocate space for the array
x_arr = np.zeros(shape=x_dims)
y_arr = np.zeros(shape=y_dims)

for i, twit in enumerate(x_bool):
    x_arr[i] = twit

for i, nchar in enumerate(y_bool):
    y_arr[i] = nchar

In [509]:
# finally, reshape into a (m, w, c) array
# where m is training example, w is window size,
# c is one-hot encoded character

x_fin = x_arr.reshape(x_arr.shape[0] * x_arr.shape[1], x_arr.shape[2], x_arr.shape[3])

# y is a (m, c) array, where m is training example and c is one-hot encoded character

y_fin = y_arr.reshape (y_arr.shape[0] * y_arr.shape[1], y_arr.shape[2])

In [510]:
print ("x_fin shape: ", x_fin.shape)
print ("y_fin shape: ", y_fin.shape)

x_fin shape:  (80, 40, 78)
y_fin shape:  (80, 78)
